### Ejemplo básico utilizando el modelo TinyBERT de Huawei

In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import warnings
import logging
# Ignorar advertencias
warnings.filterwarnings("ignore")
# Ajustar el nivel de logging para Transformers
logging.getLogger("transformers").setLevel(logging.ERROR)

# Cargar el tokenizer y el modelo TinyBERT de Huawei
model_name = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Función para predecir el sentimiento
def predict_sentiment(text):
    # Tokenización y preparación de los datos
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # # Mostrar tokens
    tokens = tokenizer.tokenize(text)
    print(f"Tokens de la oración: {tokens}")
    # Predicción
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Obtener los logits y la clase con mayor probabilidad
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    
    # Interpretar el resultado
    sentiment = "Positivo" if predicted_class == 1 else "Negativo"
    return sentiment, predicted_class

# Ejemplo de comentarios
comments = [
    "I absolutely love this product! It works wonderfully.",
    "I'm not satisfied with the service; it's too slow.",
    "Excellent quality; I would recommend it to everyone.",
    "Terrible experience; I will never buy here again."
]

# Predecir el sentimiento de cada comentario
for comment in comments:
    sentiment, class_index = predict_sentiment(comment)
    print(f"Comentario: {comment} -> Sentimiento: {sentiment} (Índice de clase: {class_index})")


Tokens de la oración: ['i', 'absolutely', 'love', 'this', 'product', '!', 'it', 'works', 'wonderful', '##ly', '.']
Comentario: I absolutely love this product! It works wonderfully. -> Sentimiento: Positivo (Índice de clase: 1)
Tokens de la oración: ['i', "'", 'm', 'not', 'satisfied', 'with', 'the', 'service', ';', 'it', "'", 's', 'too', 'slow', '.']
Comentario: I'm not satisfied with the service; it's too slow. -> Sentimiento: Positivo (Índice de clase: 1)
Tokens de la oración: ['excellent', 'quality', ';', 'i', 'would', 'recommend', 'it', 'to', 'everyone', '.']
Comentario: Excellent quality; I would recommend it to everyone. -> Sentimiento: Positivo (Índice de clase: 1)
Tokens de la oración: ['terrible', 'experience', ';', 'i', 'will', 'never', 'buy', 'here', 'again', '.']
Comentario: Terrible experience; I will never buy here again. -> Sentimiento: Negativo (Índice de clase: 0)


model.safetensors:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

In [3]:
# Imprimir detalles del modelo
print("Detalles del Modelo:")
print("---------------------")
print(f"Nombre del Modelo: {model_name}")
print(f"Número de capas: {model.bert.encoder.layer.__len__()}")
print(f"Tamaño del vector embedding: {model.bert.embeddings.word_embeddings.embedding_dim}")
print(f"Tamaño del vocabulario del modelo: {model.bert.embeddings.word_embeddings.num_embeddings}")
print(f"Número de parámetros del modelo: {sum(p.numel() for p in model.parameters())}")
print(f"Número de parámetros entrenables: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")


Detalles del Modelo:
---------------------
Nombre del Modelo: huawei-noah/TinyBERT_General_4L_312D
Número de capas: 4
Tamaño del vector embedding: 312
Tamaño del vocabulario del modelo: 30522
Número de parámetros del modelo: 14350874
Número de parámetros entrenables: 14350874


In [ ]:
# Imprimir detalles del modelo
print("Detalles del Modelo:")
print("---------------------")
print(f"Nombre del Modelo: {model_name}")

# Imprimir parámetros de la capa de embedding
embedding_params = sum(p.numel() for p in model.bert.embeddings.parameters())
print(f"Parámetros de la capa de embedding: {embedding_params} (Tipo: Embedding)")
print(" Esta capa se encarga de convertir los tokens en vectores de embeddings")
# Imprimir la cantidad de parámetros por capa de codificación
print("\nCantidad de parámetros por capa de codificación:")
total_encoder_params = 0
for i, layer in enumerate(model.bert.encoder.layer):
    num_params = sum(p.numel() for p in layer.parameters())
    layer_type = layer.__class__.__name__  # Obtener el nombre de la clase
    print(f"Capa {i + 1} ({layer_type}): {num_params} parámetros")
    total_encoder_params += num_params
print("Procesan las representaciones de los embeddings utilizando self-attention")
print("y transformers para capturar relaciones contextuales entre las palabras.")

# Imprimir parámetros de la capa de clasificación
classification_params = sum(p.numel() for p in model.classifier.parameters())
classification_type = model.classifier.__class__.__name__  # Obtener el nombre de la clase
print(f"\nParámetros de la capa de clasificación: {classification_params} (Tipo: {classification_type})")

# Número total de parámetros
total_params = embedding_params + total_encoder_params + classification_params
print(f"\nNúmero total de parámetros del modelo: {total_params}")


Detalles del Modelo:
---------------------
Nombre del Modelo: huawei-noah/TinyBERT_General_4L_312D
Parámetros de la capa de embedding: 9683856 (Tipo: Embedding)
 Esta capa se encarga de convertir los tokens en vectores de embeddings

Cantidad de parámetros por capa de codificación:
Capa 1 (BertLayer): 1142184 parámetros
Capa 2 (BertLayer): 1142184 parámetros
Capa 3 (BertLayer): 1142184 parámetros
Capa 4 (BertLayer): 1142184 parámetros
Procesan las representaciones de los embeddings utilizando self-attention
y transformers para capturar relaciones contextuales entre las palabras.

Parámetros de la capa de clasificación: 626 (Tipo: Linear)

Número total de parámetros del modelo: 14253218


#### EJEMPLO CON Distilbert pre-entrenado

In [10]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Cargar el modelo y el tokenizer preentrenados para análisis de sentimientos
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

# Función para predecir el sentimiento
def predict_sentiment(text):
    # Tokenizar el texto de entrada
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Realizar la inferencia
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Obtener la clase predicha
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    
    # Interpretar el resultado
    sentiment = "Positive" if predicted_class == 1 else "Negative"
    return sentiment, predicted_class

# Comentarios de ejemplo para analizar
comments = [
    "I absolutely love this product! It works wonderfully.",
    "I'm not satisfied with the service; it's too slow.",
    "Excellent quality; I would recommend it to everyone.",
    "Terrible experience; I will never buy here again."
]

# Analizar el sentimiento de cada comentario
for comment in comments:
    sentiment, class_index = predict_sentiment(comment)
    print(f"Comment: {comment} -> Sentiment: {sentiment} (Class index: {class_index})")



Comment: I absolutely love this product! It works wonderfully. -> Sentiment: Positive (Class index: 1)
Comment: I'm not satisfied with the service; it's too slow. -> Sentiment: Negative (Class index: 0)
Comment: Excellent quality; I would recommend it to everyone. -> Sentiment: Positive (Class index: 1)
Comment: Terrible experience; I will never buy here again. -> Sentiment: Negative (Class index: 0)


#### Detalles del modelo DistillBERT pre-entrenado

In [11]:
# Mostrar el tipo y la cantidad de capas del modelo
print("Detalles del Modelo:")
print("---------------------")

# 1. Capas de Embeddings
embedding_params = sum(p.numel() for p in model.distilbert.embeddings.parameters())
embedding_layer = model.distilbert.embeddings
embedding_length = embedding_layer.word_embeddings.weight.shape[1]  # Largo del embedding
print(f"Capa de Embeddings: {embedding_params} parámetros (Largo del embedding: {embedding_length})")

# 2. Capas de Codificación
total_encoder_params = 0
for i, layer in enumerate(model.distilbert.transformer.layer):
    num_params = sum(p.numel() for p in layer.parameters())
    layer_type = layer.__class__.__name__
    print(f"Capa {i + 1} ({layer_type}): {num_params} parámetros")
    total_encoder_params += num_params

# 3. Capa de Clasificación
classification_params = sum(p.numel() for p in model.classifier.parameters())
print(f"Capa de Clasificación: {classification_params} parámetros (Tipo: {model.classifier.__class__.__name__})")

# Número total de parámetros
total_params = embedding_params + total_encoder_params + classification_params
print(f"\nNúmero total de parámetros del modelo: {total_params}")

Detalles del Modelo:
---------------------
Capa de Embeddings: 23835648 parámetros (Largo del embedding: 768)
Capa 1 (TransformerBlock): 7087872 parámetros
Capa 2 (TransformerBlock): 7087872 parámetros
Capa 3 (TransformerBlock): 7087872 parámetros
Capa 4 (TransformerBlock): 7087872 parámetros
Capa 5 (TransformerBlock): 7087872 parámetros
Capa 6 (TransformerBlock): 7087872 parámetros
Capa de Clasificación: 1538 parámetros (Tipo: Linear)

Número total de parámetros del modelo: 66364418
